# Python DB API

## PyODBC

### SQL Server Connection

In [ ]:
#pip install pyodbc

In [1]:
import pyodbc

In [3]:
def create_server_connection(driver, server, database, user, password):
    conn = None
    try: 
        conn = pyodbc.connect(DRIVER = driver, SERVER= server, DATABASE = database, UID= user, PWD= password)
        print("Successfull")
    except Error as err:
        print(f"Error:'{err}'")
    return conn

In [ ]:
driver = '{ODBC Driver 17 for SQL Server}'
server = 'localhost' #şu anda server her birimizin kendi makinesi
database = 'master'#yeni bir db oluşturacağız
user = 'sa' 
password = '------'
conn = create_server_connection(driver, server, database, user, password)

In [3]:
pyodbc.drivers()

['SQL Server',
 'SQL Server Native Client 11.0',
 'SQL Server Native Client RDA 11.0',
 'ODBC Driver 13 for SQL Server',
 'ODBC Driver 17 for SQL Server',
 'PostgreSQL ANSI(x64)',
 'PostgreSQL Unicode(x64)']

In [4]:
create_db_syntax = 'CREATE DATABASE SAMPLEDB_1'

### Create Database

In [3]:
conn_string = "driver={ODBC Driver 17 for SQL Server}; server=localhost; database=master; TRUSTED_CONNECTION=yes;"
conn = pyodbc.connect (conn_string)
crs = conn.cursor()
conn.autocommit=True


In [2]:
conn_string = "driver={ODBC Driver 17 for SQL Server}; server=localhost; database=master; TRUSTED_CONNECTION=yes;"
conn = pyodbc.connect (conn_string)
crs = conn.cursor()
conn.autocommit=True


create_db_syntax = 'CREATE DATABASE SAMPLEDB_1'

def create_database (conn, create_db_syntax):
    conn = conn.cursor()
    try:
        conn.execute(create_db_syntax)
        print("Database is Created Succesfully")
    except Exception as err:
        print(f"Error:'{err}'")
        
create_database(conn, create_db_syntax)

Error:'('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Database 'SAMPLEDB_1' already exists. Choose a different database name. (1801) (SQLExecDirectW)")'


In [11]:
def execute_query(conn, query):
    conn = conn.cursor()
    try:
        conn.execute(query)
        conn.commit()
        print("Query Succeessful!")
    except Error as err:
        print(f"Error:'{err}'")

In [12]:
query = 'USE SAMPLEDB_1'

In [13]:
execute_query(conn, query)

Query Succeessful!


### Create table

In [9]:
query = 'CREATE TABLE TestA (\
           ID INT IDENTITY (1,1) NOT NULL, \
           FirstName VARCHAR(255) NOT NULL,\
           LastName VARCHAR(255) NOT NULL,\
           PRIMARY KEY(ID))'

In [10]:
execute_query(conn, query)

Query Succeessful!


### Insert

In [11]:
crs.execute("INSERT TestA (FirstName, LastName) VALUES ('Bob', 'Marley')")
#conn.commit()

In [12]:
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Bruce', 'Lee')

In [13]:
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Tom', 'Hanks')
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Bart', 'Simpson')

In [14]:
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Tom', 'Cat')
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Jerry', 'Mouse')
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Owen', 'William')
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Stefan', 'Müller')

### Delete

In [15]:
crs.execute("DELETE FROM TestA WHERE FirstName = 'Jerry'")

In [16]:
crs.execute("DELETE FROM TestA WHERE FirstName = ?" ,'Stefan')

### Rowcount

In [18]:
crs.rowcount

1

In [19]:
print(crs.rowcount,"row(s) deleted")

1 row(s) deleted


## Reading Data

### Fetchone

In [9]:
crs.execute('SELECT FirstName, LastName FROM TestA')

ProgrammingError: ('42S02', "[42S02] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Invalid object name 'TestA'. (208) (SQLExecDirectW)")

In [21]:
row = crs.fetchone() #tablodaki ilk kaydı getirir
row

('Bob', 'Marley')

In [22]:
row = crs.fetchone() #tablodaki ilk kaydı getirir
row

('Bruce', 'Lee')

In [24]:
row.FirstName

'Bruce'

In [25]:
row.LastName

'Lee'

### Fetchall

In [26]:
crs.execute('SELECT ID, FirstName, LastName FROM TestA')
rows = crs.fetchall()
rows

[(1, 'Bob', 'Marley'),
 (2, 'Bruce', 'Lee'),
 (3, 'Tom', 'Hanks'),
 (4, 'Bart', 'Simpson'),
 (5, 'Tom', 'Cat'),
 (7, 'Owen', 'William')]

In [27]:
for row in rows:
    print(row.ID, row.FirstName, row.LastName)

1 Bob Marley
2 Bruce Lee
3 Tom Hanks
4 Bart Simpson
5 Tom Cat
7 Owen William


In [28]:
crs.execute("""SELECT ID, FirstName, LastName 
                FROM TestA""")
rows = crs.fetchall()
rows

[(1, 'Bob', 'Marley'),
 (2, 'Bruce', 'Lee'),
 (3, 'Tom', 'Hanks'),
 (4, 'Bart', 'Simpson'),
 (5, 'Tom', 'Cat'),
 (7, 'Owen', 'William')]

In [29]:
crs.execute("""SELECT ID, FirstName, LastName 
               FROM TestA
               WHERE FirstName = ? and LastName = ?""", ['Owen', 'William'])
rows = crs.fetchall()
rows

[(7, 'Owen', 'William')]

### Fetchmany

In [30]:
crs.execute("""SELECT ID, FirstName, LastName 
               FROM TestA""")
rows = crs.fetchmany(2)
rows

[(1, 'Bob', 'Marley'), (2, 'Bruce', 'Lee')]

### Fetchval

In [31]:
crs.execute("SELECT MAX(ID) FROM TestA").fetchval()

7

In [32]:
crs.execute("SELECT FirstName FROM TestA").fetchval()

'Bob'

In [33]:
crs.execute("SELECT FirstName, LastName FROM TestA").fetchval()

'Bob'

# Pandas Methods

In [20]:
import pandas as pd

In [6]:
def read_query (conn, query): #connection nesnesi yukarıda tanımlanmıştı.
    crs = conn. cursor()
    results = None
    crs.execute(query)
    results = crs.fetchall()
    return results

In [15]:
query = """SELECT ID, FirstName, LastName 
               FROM TestA"""

In [16]:
read_query(conn, query)

[(1, 'Bob', 'Marley'),
 (2, 'Bruce', 'Lee'),
 (3, 'Tom', 'Hanks'),
 (4, 'Bart', 'Simpson'),
 (5, 'Tom', 'Cat'),
 (7, 'Owen', 'William')]

In [25]:
result = read_query(conn, query)

In [26]:
result

[(1, 'Bob', 'Marley'),
 (2, 'Bruce', 'Lee'),
 (3, 'Tom', 'Hanks'),
 (4, 'Bart', 'Simpson'),
 (5, 'Tom', 'Cat'),
 (7, 'Owen', 'William')]

In [27]:
df = pd.DataFrame(result)

In [28]:
df

,0
0,"[1, Bob, Marley]"
1,"[2, Bruce, Lee]"
2,"[3, Tom, Hanks]"
3,"[4, Bart, Simpson]"
4,"[5, Tom, Cat]"
5,"[7, Owen, William]"


In [31]:
from_db = []

for result in result:
    result = list(result)
    from_db.append(result)

In [32]:
from_db

[[1, 'Bob', 'Marley'],
 [2, 'Bruce', 'Lee'],
 [3, 'Tom', 'Hanks'],
 [4, 'Bart', 'Simpson'],
 [5, 'Tom', 'Cat'],
 [7, 'Owen', 'William']]

In [33]:
columns = ['ID','FirstName', 'LastName']
df = pd.DataFrame(from_db, columns = columns)
df

,ID,FirstName,LastName
0,1,Bob,Marley
1,2,Bruce,Lee
2,3,Tom,Hanks
3,4,Bart,Simpson
4,5,Tom,Cat
5,7,Owen,William


In [34]:
query = 'USE SampleSales'
execute_query(conn,query)

Query Succeessful!


In [35]:
crs.execute("SELECT * FROM product.brand").fetchall()

[(1, 'Electra'),
 (2, 'Haro'),
 (3, 'Redline'),
 (4, 'Cannondale'),
 (5, 'Schwinn'),
 (6, 'Giant'),
 (7, 'Sun Bicycles'),
 (8, 'Surly'),
 (9, 'Trek')]

## read_sql

In [36]:
pd.read_sql("SELECT * FROM product.brand", con = conn)

,brand_id,brand_name
0,1,Electra
1,2,Haro
2,3,Redline
3,4,Cannondale
4,5,Schwinn
5,6,Giant
6,7,Sun Bicycles
7,8,Surly
8,9,Trek


In [37]:
df_prod = pd.read_sql("SELECT * FROM product.product", con = conn)
df_prod.head(4)

,product_id,product_name,brand_id,category_id,model_year,list_price
0,1,Trek 820 - 2018,9,6,2018,379.99
1,2,Schwinn Suburban DLX Step-Thru - 2018,5,3,2018,749.99
2,3,Surly Wednesday Frameset - 2018,8,6,2018,999.99
3,4,Trek Fuel EX 8 29 - 2018,9,6,2018,2899.99


## python to SQL 

In [38]:
import pyodbc
import pandas as pd

In [ ]:
import pyodbc
import pandas as pd

In [39]:
df = pd.read_csv("IMDB top 1000_LV.csv")
df.head(2)

,Movie_ID,Title,Certificate,Duration,Genre,Rate,Metascore,Description,Year,Directors,Stars,Votes,Gross($M)
0,1,The Shawshank Redemption,13+,142,Drama,9.3,80,Two imprisoned men bond over a number of years...,1994,Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",2377444,28.34
1,2,The Godfather,16,175,"Crime, Drama",9.2,100,An organized crime dynasty's aging patriarch t...,1972,Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Diane Ke...",1646788,134.97


In [40]:
records = df.to_dict('records')

In [41]:
records

[{'Movie_ID': 1,
  'Title': 'The Shawshank Redemption',
  'Certificate': '13+',
  'Duration': 142,
  'Genre': 'Drama',
  'Rate': 9.3,
  'Metascore': 80,
  'Description': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.',
  'Year': 1994,
  'Directors': 'Frank Darabont',
  'Stars': 'Tim Robbins, Morgan Freeman, Bob Gunton, William Sadler',
  'Votes': 2377444,
  'Gross($M)': 28.34},
 {'Movie_ID': 2,
  'Title': 'The Godfather',
  'Certificate': '16',
  'Duration': 175,
  'Genre': 'Crime, Drama',
  'Rate': 9.2,
  'Metascore': 100,
  'Description': "An organized crime dynasty's aging patriarch transfers control of his clandestine empire to his reluctant son.",
  'Year': 1972,
  'Directors': 'Francis Ford Coppola',
  'Stars': 'Marlon Brando, Al Pacino, James Caan, Diane Keaton',
  'Votes': 1646788,
  'Gross($M)': 134.97},
 {'Movie_ID': 3,
  'Title': 'The Dark Knight',
  'Certificate': '16',
  'Duration': 152,
  'Genre': 'A

In [42]:
query = """
        CREATE TABLE imdb (
            Movie_ID INTEGER primary key,
            Title VARCHAR(255) NOT NULL,
            Certificate VARCHAR(255) NOT NULL,
            Duration INTEGER NOT NULL,
            Genre VARCHAR(255) NOT NULL,
            Rate FLOAT NOT NULL,
            Metascore INTEGER,
            Description VARCHAR(4096) NOT NULL,
            Year INTEGER NOT NULL,
            Directors VARCHAR(255) NOT NULL,
            Stars VARCHAR(255) NOT NULL,
            Votes INTEGER,
            Gross FLOAT
        )
        """
execute_query(conn, query)

Query Succeessful!


In [43]:
query = 'INSERT imdb VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)'

In [44]:
for k,v in enumerate(records, start = 1):
    crs.execute(query, (k, v['Title'], v['Certificate'], v['Duration'], v['Genre'], v['Rate'], v['Metascore'],v['Description'], v['Year'], v['Directors'], v['Stars'], v['Votes'], v['Gross($M)']))